## @property

https://www.freecodecamp.org/news/python-property-decorator/

In [13]:
class House1:

    def __init__(self, price):
        self.price = price

### defining getters, setters

In [27]:
class House:
     
    #make _price private
    def __init__(self, price):
        self._price = price
    
    # @property same as defining price.getter
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, new_price):
        if new_price > 0 and isinstance(new_price, float):
            self._price = new_price
        else:
            print('please enter valid price')
        
    @price.deleter
    def price(self):
        del self._price


- @price.setter - Used to indicate that this is the setter method for the price property. Notice that we are not using @property.setter, we are using @price.setter. The name of the property is included before .setter.

using

### testing

In [28]:
house = House(5000)
house.price

5000

In [29]:
house.price =-50

please enter valid price


In [30]:
del house.price

## @Classmethod

A class method is a method that is bound to a class rather than its object. It doesn't require creation of a class instance, much like staticmethod.

The difference between a static method and a class method is:
- Static method knows nothing about the class and just deals with the parameters
- Class method works with the class since its parameter is always the class itself.

But no matter what, the class method is always attached to a class with the first argument as the class itself cls.

In [ ]:
def classMethod(cls, args...)

In [37]:
class Person:
    age = 25
    
    # We also have a function printAge that takes a single parameter cls and not self we usually take.
    @classmethod
    def printAge(cls):
        print('The age is:', cls.age)

# In the final line, we call printAge without creating a Person object like we do for static methods. 
#This prints the class variable age.
Person.printAge()

The age is: 25


without creating instance (i.e. person = Person()), can call .printAge()

### when to use

#### factory method

Factory methods are those methods that return a class object (like constructor) for different use cases. 
like function overloading

In [33]:
from datetime import date

# random Person
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def fromBirthYear(cls, name, birthYear):
        return cls(name, date.today().year - birthYear)

    def display(self):
        print(self.name + "'s age is: " + str(self.age))

person = Person('Adam', 19)
person.display()

person1 = Person.fromBirthYear('John',  1985)
person1.display()

Adam's age is: 19
John's age is: 36


#### ensure correct instance creation in inheritance 

Whenever you derive a class from implementing a factory method as a class method, it ensures correct instance creation of the derived class.

You can create a static method for the above example but the object it creates, will always be hardcoded as Base class.

In [61]:
from datetime import date

# random Person
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @staticmethod
    def fromFathersAge(name, fatherAge, fatherPersonAgeDiff):
        return Person(name, date.today().year - fatherAge + fatherPersonAgeDiff)

    @classmethod
    def fromBirthYear(cls, name, birthYear):
        return cls(name, date.today().year - birthYear)

    def display(self):
        print(self.name + "'s age is: " + str(self.age))

class Man(Person):
    sex = 'Male'

# classmethod good
man = Man.fromBirthYear('John', 1985)
print(isinstance(man, Man))

# violate OOP. return Person's object instead of Man's object
man1 = Man.fromFathersAge('John', 1965, 20)
print(isinstance(man1, Man))
print(isinstance(man1, Person))

True
False
True


Here, using a static method to create a class instance wants us to hardcode the instance type during creation.

This clearly causes a problem when inheriting Person to Man.

fromFathersAge method doesn't return a Man object but its base class Person's object.

This violates OOP paradigm. Using a class method as fromBirthYear can ensure the OOP-ness of the code since it takes the first parameter as the class itself and calls its factory method.

## @Staticmethod

Static methods have limited use, because they don't have access to the attributes of an instance of a class (like a regular method does), and they don't have access to the attributes of the class itself (like a class method does).

However, they can be useful to group some utility function together with a class - e.g. a simple conversion from one type to another - that doesn't need access to any information apart from the parameters provided (and perhaps some attributes global to the module.)

They could be put outside the class, but grouping them inside the class may make sense where they are only applicable there.

#### group utility function to class

Static methods have a limited use case because, like class methods or any other methods within a class, they cannot access the properties of the class itself.

However, when you need a utility function that doesn't access any properties of a class but makes sense that it belongs to the class, we use static functions.

In [67]:
class Dates:
    def __init__(self, date):
        self.date = date
        
    def getDate(self):
        return self.date

    @staticmethod
    def toDashDate(date):
        return date.replace("/", "-")

    
date = Dates("15-12-2016")
dateFromDB = "15/12/2016"
dateWithDash = Dates.toDashDate(dateFromDB)

if(date.getDate() == dateWithDash):
    print("Equal")
    
else:
    print("Unequal")

Equal


toDashDate is a static method because it doesn't need to access any properties of Dates itself and only requires the parameters.

We can also create toDashDate outside the class, but since it works only for dates, it's logical to keep it inside the Dates class.

#### have single implementation

Static methods are used when we don't want subclasses of a class change/override a specific implementation of a method.

In [64]:
class Dates:
    def __init__(self, date):
        self.date = date
        
    def getDate(self):
        return self.date

    @staticmethod
    def toDashDate(date):
        return date.replace("/", "-")

class DatesWithSlashes(Dates):
    def getDate(self):
        return Dates.toDashDate(self.date)

date = Dates("15-12-2016")
dateFromDB = DatesWithSlashes("15/12/2016")

if(date.getDate() == dateFromDB.getDate()):
    print("Equal")
else:
    print("Unequal")

Equal


Here, we wouldn't want the subclass DatesWithSlashes to override the static utility method toDashDate because it only has a single use, i.e. change date to dash-dates.

We could easily use the static method to our advantage by overriding getDate() method in the subclass so that it works well with the DatesWithSlashes class.